In [1]:
import pandas as pd
import altair as alt
import pulp  # pip install pulp
import numpy as np
from scipy.ndimage.filters import convolve
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class SeamCarve:

    def energy(self, image):
        """
        Computes the "energy" of an image.

        Parameters
        ----------
        image : numpy.ndarray
            A colour image

        Returns
        -------
        numpy.ndarray
            A new image where the pixels values represent the energy
            of the corresponding pixel in the original image
        """

        dy = np.array([-1, 0, 1])[:, None, None]
        dx = np.array([-1, 0, 1])[None, :, None]
        energy_img = convolve(image, dx)**2 + convolve(image, dy)**2
        return np.sum(energy_img, axis=2)

    def find_vertical_seam(self, energy):
        """
        PLACEHOLDER FUNCTION - This will be replaced further on in the lab.  

        Parameters
        ----------
        energy : numpy.ndarray
            the 2d energy image

        Returns
        -------
        numpy.ndarray
            an array containing all zeros
        """

        return np.zeros(energy.shape[0])

    def find_horizontal_seam(self, energy):
        """
        Find the minimum-energy horizontal seam in an image. 

        Parameters
        ----------
        energy : numpy.ndarray
            a 2d numpy array containing the energy values. Size NxM

        Returns
        -------
        numpy.ndarray
            a seam represented as a 1d array of length M, with all values between 0 and N-1
        """

        return self.find_vertical_seam(energy.T)

    def remove_vertical_seam(self, image, seam):
        """
        Remove a vertical seam from an image.

        Parameters
        ----------
        image : numpy.ndarray
            a 3d numpy array containing the pixel values
        seam : numpy.ndarray
            a 1d array (or list) containing the column index of each pixel in the seam
            length N, all values between 0 and M-1

        Returns
        -------
        numpy.ndarray
            a new image that is smaller by 1 column. Size N by M-1.
        """

        height = image.shape[0]
        linear_inds = np.array(seam)+np.arange(image.shape[0])*image.shape[1]
        new_image = np.zeros(
            (height, image.shape[1]-1, image.shape[-1]), dtype=image.dtype)
        for c in range(image.shape[-1]):
            temp = np.delete(image[:, :, c], linear_inds.astype(int))
            temp = np.reshape(temp, (height, image.shape[1]-1))
            new_image[:, :, c] = temp
        return new_image

    # Same as remove_vertical_seam above, but for a horizontal seam. The output image is size N-1 by M.
    def remove_horizontal_seam(self, image, seam):
        """
        Remove a horizontal seam from an image.

        Parameters
        ----------
        image : numpy.ndarray 
            a 2d numpy array containing the pixel values. Size NxM
        seam : numpy.ndarray
            a 1d array containing the column index of each pixel in the seam
            length N, all values between 0 and M-1.

        Returns
        -------
        numpy.ndarray
            a new image that is smaller by 1 row. Size N-1 by M.
        """

        return np.transpose(self.remove_vertical_seam(np.transpose(image, (1, 0, 2)), seam), (1, 0, 2))

    def seam_carve(self, image, desired_height, desired_width):
        """
        Resize an NxM image to a desired height and width. 
        Note: this function only makes images smaller. Enlarging an image is not implemented. 
        Note: this function removes all vertical seams before removing any horizontal
        seams, which may not be optimal.

        Parameters
        ----------
        image : numpy.ndarray
            a 3d numpy array of size N x M x 3
        desired_width : int 
            the desired width
        desired_height : int 
            the desired height

        Returns
        -------
        numpy array
            the resized image, now of size desired_height x desired_width x 3

        Example
        ------- 
        >>> sc = SeamCarve()
        >>> image = np.array([[[0.2 , 0.1 , 0.6 ], [0.4 , 0.5 , 0.65]], [[0.3 , 0.7 , 0.3 ], [0.8 , 0.33, 0.7 ]]])
        >>> sc.seam_carve(image, 1, 1 )
        array([[[0.8 , 0.33, 0.7 ]]])
        """

        while image.shape[1] > desired_width:
            seam = self.find_vertical_seam(self.energy(image))
            assert len(
                seam) == image.shape[0], "the length of the seam must equal the height of the image"
            image = self.remove_vertical_seam(image, seam)
            print('\nWidth is now %d' % image.shape[1], end='')
        print()
        while image.shape[0] > desired_height:
            seam = self.find_horizontal_seam(self.energy(image))
            assert len(
                seam) == image.shape[1], "the length of the seam must equal the width of the image"
            image = self.remove_horizontal_seam(image, seam)
            print('\nHeight is now %d' % image.shape[0], end='')
        print()
        return image

recursive implementation¶

In [ ]:
def fvs(energy, seam, cost):
    """
    The "inner" recursive function for finding a vertical seam.

    Parameters
    ----------
    energy : numpy.ndarray
        the 2d energy image
    seam : list
        the partial seam (from the top, partway down the image)
    cost : float
        the cost of the partial seam

    Returns
    -------
    tuple 
        contains the seam and the energy cost

    Example
    -------    
    >>> fvs(np.array([[0.6625, 0.3939], [1.0069, 0.7383]]), [1], 0.0)
    ([1, 1], 1.1321999999999999)
    """

    row = len(seam)-1
    col = seam[-1]

    # if out of bounds on one of the two sides, return infinite energy
    if col < 0 or col >= energy.shape[1]:
        return seam, np.inf

    cost += energy[row, col]

    # regular base case: reached bottom of image
    # your code here
    if len(seam) == energy.shape[0]:
        return seam, cost
    # your code here

    return min((fvs(energy, seam+[col], cost),
                fvs(energy, seam+[col+1], cost),
                fvs(energy, seam+[col-1], cost)), key=lambda x: x[1])


class SeamCarveRecursive(SeamCarve):

    def find_vertical_seam(self, energy):
        """
        Finds the vertical seam of lowest total energy inside the image.

        Parameters
        ----------
        energy : numpy.ndarray
            the 2d energy image

        Returns
        -------
        list 
            the seam of column indices

        Example
        -------  
        >>> sc_recursive = SeamCarveRecursive()
        >>> e = np.array([[0.6625, 0.3939], [1.0069, 0.7383]])
        >>> sc_recursive.find_vertical_seam(e)
        (1, 1)
        """
        
        costs = dict()
        for i in range(energy.shape[1]):
            best_seam, best_cost = fvs(energy, [i], 0.0)
            costs[tuple(best_seam)] = best_cost
        return min(costs, key=costs.get)  # the best out of the M best seams

#### integer linear program

In [ ]:
class SeamCarveILP(SeamCarve):

    def find_vertical_seam(self, energy):
        """
        Finds the vertical seam of lowest total energy inside the image.

        Parameters
        ----------
        energy : numpy.ndarray
            the 2d energy image

        Returns
        -------
        list 
            the seam of column indices

        Example
        -------    
        >>> sc = SeamCarveILP()
        >>> e = np.array([[0.6625, 0.3939], [1.0069, 0.7383]])
        sc.find_vertical_seam(e)
        [1, 1]
        """ 
            
        N, M = energy.shape

        # initialize the optimization problem, give it a name
        prob = pulp.LpProblem("Seam carving", pulp.LpMinimize)

        # create the x_ij variables
        x = pulp.LpVariable.dicts("pixels",(list(range(N)),list(range(M))),0,1,pulp.LpInteger)

        # The objective function is being built here. The objective is the sum of energies in the seam.
        objective_terms = list()
        for i in range(N):
            for j in range(M):
                objective_terms.append(energy[i,j]*x[i][j])
        prob += pulp.lpSum(objective_terms) # adds up all the terms in the list

        # Constraint #1: one pixel per row
        
        # your code here
        for i in range(N):
            prob += pulp.lpSum([x[i][j] for j in range(M)]) == 1 # 1 pixel per row
        # your code here

        # Constraint #2: connectivity of seam
        for i in range(N-1):
            for j in range(M): # below: this says: x(i,j) - x(i+1,j-1) - x(i+1,j) - x(i+1,j+1) <= 0
                prob += pulp.lpSum([x[i][j]]+[-x[i+1][k] for k in range(max(0,j-1),min(M,j+2))]) <= 0

        # Solve the problem
        prob.solve()

        # Build up the seam by collecting the pixels in the optimal seam
        # Note: you can access the values (0 or 1) of the variables with pulp.value(x[i][j])
        # your code here
        seam = []
        for i in range(N):
            for j in range(M):
                if pulp.value(x[i][j])==1:
                    seam.append(j)
                    break        
        # your code here

        return seam

In [ ]:
#### dynamic programming implementation

In [2]:
class SeamCarveDP(SeamCarve):

    def find_vertical_seam(self, energy):
        """
        The vertical seam of lowest total energy inside the image.

        Parameters
        ----------
        energy : numpy.ndarray
            the 2d energy image

        Returns
        -------
        list 
            the seam of column indices

        Example
        -------    
        >>> sc = SeamCarveDP()
        >>> e = np.array([[0.6625, 0.3939], [1.0069, 0.7383]])
        sc.find_vertical_seam(e)
        [1, 1]     
        """
        
        nrows, ncols = energy.shape

        # Cumulative Minimum Energy setup
        CME = np.zeros((nrows, ncols+2))
        CME[:,0] = np.inf
        CME[:,-1] = np.inf
        CME[:,1:-1] = energy
        
        
        # calculate the Cumulative Minimum Energies
        #your code here
        for i in range(1,nrows):
            for j in range(1,ncols+1):
                CME[i,j] += min(CME[i-1,j-1],CME[i-1,j],CME[i-1,j+1])

        #your code here

        # create the seam array
        seam = np.zeros(nrows, dtype=int)
        seam[-1] = np.argmin(CME[-1,:])

        # track the path backwards
        for i in range(nrows-2,-1,-1):
            # min_index is 0, 1, or 2. I subtract 1 to give the offset from
            # seam(i+1), namely -1, 0, or 1. Then I add this to the old value.
            delta = np.argmin(CME[i, seam[i+1]-1:seam[i+1]+2]) - 1
            seam[i] =  seam[i+1] + delta

        return seam - 1 
        # Above: -1 because the indices are all off by 1, due to padding of CME
        #        This has nothing to do with the -1 in defining "delta'".

NameError: name 'SeamCarve' is not defined

In [ ]:
#### code vectorization

In [ ]:
class SeamCarveDPVec(SeamCarve):

    def find_vertical_seam(self, energy):
        """
        The vertical seam of lowest total energy inside the image.

        Parameters
        ----------
        energy : numpy.ndarray
            the 2d energy image

        Returns
        -------
        list 
            the seam of column indices

        Example
        -------    
        >>> sc = SeamCarveDPVec()
        >>> e = np.array([[0.6625, 0.3939], [1.0069, 0.7383]])
        sc.find_vertical_seam(e)
        [1, 1]     
        """

        nrows, ncols = energy.shape

        # Cumulative Minimum Energy
        CME = np.zeros((nrows, ncols+2))
        CME[:, 0] = np.inf
        CME[:, -1] = np.inf
        CME[:, 1:-1] = energy
        
        # your code here
        for i in range(1, nrows):
            options = np.vstack((CME[i-1, 0:-2], CME[i-1, 1:-1], CME[i-1, 2:]))
            CME[i, 1:-1] += np.min(options, axis=0)            
        # your code here
        
        # create the seam array
        seam = np.zeros(nrows, dtype=int)
        seam[-1] = np.argmin(CME[-1, :])

        # track the path backwards
        for i in range(nrows-2, -1, -1):
            # min_index is 0, 1, or 2. I subtract 1 to give the offset from
            # seam(i+1), namely -1, 0, or 1. Then I add this to the old value.
            delta = np.argmin(CME[i, seam[i+1]-1:seam[i+1]+2]) - 1
            seam[i] = seam[i+1] + delta

        return seam - 1
        # Above: -1 because the indices are all off by 1, due to padding of CME
        #        This has nothing to do with the -1 in defining "delta'".